In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("../..") if not "google.colab" in sys.modules else Path("aves_git")

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 96
# esto depende de las fuentes que tengas instaladas en el sistema.
mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"

In [ ]:
# Load
zones = (
    gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
    .set_index("ID")
)
zones.head()

In [ ]:
# Sistemas de coordenadas
zones.crs

In [ ]:
zones.total_bounds

In [ ]:
zones.plot()

In [ ]:
zones_utm = zones.to_crs('epsg:5361')
zones_utm.total_bounds

In [ ]:
zones.plot()

In [ ]:
# Grillas

In [ ]:
from aves.models.grid import S2Grid

square_grid = S2Grid.from_geodf(zones, grid_level=12)
square_grid

In [ ]:
square_grid.geodf

In [ ]:
ax = square_grid.geodf.plot(edgecolor='grey', linewidth=0.5, color='none')
zones.plot(ax=ax, color='none', edgecolor='pink', linewidth=1)

In [ ]:
from aves.models.grid import H3Grid

hex_grid = H3Grid.from_geodf(zones, grid_level=7, extra_margin=0.025)
hex_grid.geodf

In [ ]:
ax = hex_grid.geodf.plot(edgecolor='grey', linewidth=0.5, color='none')
zones.plot(ax=ax, color='none', edgecolor='pink', linewidth=1)

In [ ]:
# Cruces (Joins)

In [ ]:
from aves.data import eod

viajes = eod.read_trips(EOD_PATH).merge(eod.read_people(EOD_PATH))

viajes["PesoLaboral"] = (
    viajes["FactorLaboralNormal"] * viajes["Factor_LaboralNormal"]
)

viajes = viajes[pd.notnull(viajes["PesoLaboral"])]

viajes.columns

In [ ]:
from aves.features.geo import to_point_geodataframe

destinos_viajes = to_point_geodataframe(viajes, 'DestinoCoordX', 'DestinoCoordY', crs='epsg:5361')
destinos_viajes.plot()

In [ ]:
gpd.sjoin(destinos_viajes, zones, op='within')

In [ ]:
destinos_viajes[destinos_viajes['ComunaDestino'] == 'Las Condes'].to_crs(zones.crs).plot(aspect=1)

In [ ]:
viajes_area_interes = gpd.sjoin(destinos_viajes.to_crs(zones.crs), zones, op='within')
viajes_area_interes.plot(marker='.', markersize=1)

In [ ]:
viajes_area_interes.columns

In [ ]:
destinos_area = gpd.sjoin(viajes_area_interes.drop('index_right', axis=1), hex_grid.geodf, op='within').groupby(['h3_cell_id', 'Proposito'])['PesoLaboral'].sum().unstack(fill_value=0)

destinos_area

In [ ]:
from aves.visualization.figures import small_multiples_from_geodataframe

fig, axes = small_multiples_from_geodataframe(zones, 3)

for ax, col in zip(axes, ['Al trabajo', 'De salud', 'Recreación']):
    hex_grid.geodf.join(destinos_area.pipe(np.sqrt), on='h3_cell_id').plot(column=col, cmap='Oranges', ax=ax, legend=True)
    ax.set_title(col)

In [ ]:
# Agregaciones (Dissolve)

In [ ]:
municipios = zones.dissolve(by='NOM_COMUNA')
municipios

In [ ]:
municipios.plot()

In [ ]:
fig, axes = small_multiples_from_geodataframe(zones, 3, height=9)

for ax, col in zip(axes, ['Al trabajo', 'De salud', 'Recreación']):
    hex_grid.geodf.join(destinos_area.pipe(np.sqrt), on='h3_cell_id').plot(column=col, cmap='Oranges', ax=ax, legend=True)
    municipios.plot(ax=ax, linewidth=0.5, edgecolor='grey', color='none')
    ax.set_title(col)

In [ ]:
# Operaciones

In [ ]:
from aves.data.census import read_census_map

municipios_censo = read_census_map('COMUNA')

In [ ]:
municipios_censo.plot()

In [ ]:
hex_grid_municipios = gpd.sjoin(hex_grid.geodf, municipios_censo, op='intersects')
len(hex_grid.geodf), len(hex_grid_municipios)

In [ ]:
ax = hex_grid_municipios.plot(figsize=(12, 12), edgecolor='white', column='NOM_COMUNA', alpha=0.5)
#municipios_censo.plot(ax=ax, color='none', edgecolor='green')

In [ ]:
hex_grid_municipios_overlay = gpd.overlay(hex_grid.geodf, municipios_censo, how='intersection')
len(hex_grid.geodf), len(hex_grid_municipios_overlay)

In [ ]:
# https://geopandas.org/en/stable/docs/user_guide/set_operations.html
ax = hex_grid_municipios_overlay.plot(figsize=(12, 12), edgecolor='white', column='NOM_COMUNA', alpha=0.5)


In [ ]:
hex_grid_municipios_overlay[hex_grid_municipios_overlay['NOM_COMUNA'] == 'SANTIAGO'].plot(aspect=1)

In [ ]:
# Pesos Espaciales
from libpysal.weights import Queen

In [ ]:
from aves.features.geo import clip_area_geodataframe

municipios_area = clip_area_geodataframe(municipios_censo, zones.total_bounds).reset_index(drop=True)
municipios_area

In [ ]:
w_queen = Queen.from_dataframe(municipios_area)
w_queen.n

In [ ]:
w_queen.pct_nonzero

In [ ]:
ax = municipios_area.plot(edgecolor='grey', facecolor='w')
f,ax = w_queen.plot(municipios_area, ax=ax, 
        edge_kws=dict(color='r', linestyle=':', linewidth=1),
        node_kws=dict(marker='o'))

In [ ]:
sns.heatmap(w_queen.full()[0], xticklabels=municipios_area['NOM_COMUNA'], yticklabels=municipios_area['NOM_COMUNA'])

In [ ]:
from libpysal.weights import Kernel
# https://pysal.org/libpysal/generated/libpysal.weights.Kernel.html

w_kernel = Kernel.from_dataframe(municipios_area, k=2)
w_kernel.n

In [ ]:
w_kernel.bandwidth

In [ ]:
w_kernel.pct_nonzero

In [ ]:
ax = municipios_area.plot(edgecolor='grey', facecolor='w')
f,ax = w_kernel.plot(municipios_area, ax=ax, 
        edge_kws=dict(color='r', linestyle=':', linewidth=1),
        node_kws=dict(marker='o'))

In [ ]:
municipios_area['NOM_COMUNA']

In [ ]:
sns.heatmap(w_kernel.full()[0], xticklabels=municipios_area['NOM_COMUNA'], yticklabels=municipios_area['NOM_COMUNA'])